In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import requests
import time
import sys, os

In [63]:
def get_percentile(x, t='latest',val=0):
    if t=='latest':
        x=x.dropna().values
        val=x[-1]
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    elif t=='each':
        x=x.dropna().values
        return np.round(100*(np.where(np.sort(x)==val)[0][0]/len(x)),2)
    else:
        return  np.percentile(np.sort(x),val)

In [3]:
# RSI, %time above RSI
def RSI(series, period=14):
 #print(len(series))
 delta = series.diff().dropna()
 u = delta * 0
 d = u.copy()
 u[delta > 0] = delta[delta > 0]
 d[delta < 0] = -delta[delta < 0]
 u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
 u = u.drop(u.index[:(period-1)])
 d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
 d = d.drop(d.index[:(period-1)])
 #rs = pd.stats.moments.ewma(u, com=period-1, adjust=False) / pd.stats.moments.ewma(d, com=period-1, adjust=False)
 rs = pd.Series.ewm(u, com=period-1).mean()/pd.Series.ewm(d, com=period-1).mean()
 rsi = 100 - 100 / (1 + rs)
 return np.round(rsi.values[-1],2),get_percentile(rsi)#np.round(100*np.sum(np.where(rsi>rsi.values[-1],1,0))/len(rsi),2)

In [4]:
def consecutive(x,lag=4):
    c=0
    for i in x[-lag:][::-1]:
        if i>=0:
            c+=1
        else:
            return c
    return c    

def consecutive_increases(x):
    c=0
    latest=x.values[-1]
    for i in x[::-1]:
        if i==latest:
            continue
        if latest>i:
            c+=1
            latest=i
        else:
            return c
    return c    

In [5]:
# Consecutive increases, hist percentile, hist change mean 4 week, slope of 4 week macd
def macd_calcs(x):
    macd=x.ewm(span=12,min_periods=0,adjust=True,ignore_na=True).mean()-x.ewm(span=26,min_periods=0,adjust=True,ignore_na=True).mean()
    signal=macd.ewm(span=9,min_periods=0,adjust=True,ignore_na=True).mean()
    macd_hist=macd-signal
    return consecutive_increases(macd_hist), get_percentile(macd_hist),(macd_hist/macd_hist.shift(1)-1).rolling(4).mean().values[-1:][0],np.polyfit(x=[1,2,3,4],y=macd.values[-4:],deg=1,full=False)[0]

In [6]:
def get_intraday(ticker):
    url='https://cloud.iexapis.com/stable/stock/{}/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartLast=10&chartIEXWhenNull=True&chartInterval=10'
    d={'chartIEXOnly': False}
    response = requests.get(
        url.format(ticker),
       params=d
        )
    for i in response.json()[::-1]:
        if ((i['close'] is None)==False):
            return ([i['open'],i['high'],i['low'],i['close'],i['close']])
    url='https://cloud.iexapis.com/stable/stock/{}/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartLast=10&chartIEXWhenNull=True&chartInterval=10'
    d={'chartIEXOnly': True}
    response = requests.get(
        url.format(ticker),
       params=d
        )
    for i in response.json()[::-1]:
        if ((i['marketClose'] is None)==False):
            return([i['marketOpen'],i['marketHigh'],i['marketLow'],i['marketClose'],i['marketClose']])

In [17]:
#
holdings='NOMD,GDXJ,BGS,AWK,SHW,EAT,CF,BATT,CAKE,UAA,TTEK,NWL,FAST,FXE,DVA,URA,ITB,GOLD,SPY,DOCU,ZM,PTON,MSFT,NVDA,EA,CRM,VEEV,ADBE,QQQ,TSLA,W,ESTC,CLX,GDX,CDNS,MELI,ARKF,TTD,WK,SNAP,ESPO,NEM,YETI,CYTK,AAXN,FCX'


In [18]:
holdings=holdings.split(',')

In [117]:
#holdings=['GDX','RHHBY','ETSY','ECL','OLLI','BLK','SPY','SPXL','AZN','CDNS','CWH','BYND']
holdings=['CRM']

In [118]:
res=[[],[],[],[],[],[],[],[],[],[],[],[],[]]

In [119]:
holdings

['CRM']

In [120]:
c=1
for ticker in holdings:
    print(ticker)
    time.sleep(1)
    c+=1
    if c==4:
        c=1
        time.sleep(60)
    d={'function': 'TIME_SERIES_DAILY_ADJUSTED','apikey':"YEUC2DL0F7KC43PB",'symbol':ticker,'outputsize':'full'}
    url='https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&apikey=YEUC2DL0F7KC43PB&outputsize=full'.format(ticker)
    response = requests.get(
        url
#        params=d,
        )
    #print(response.text)
    #print(url)
    try:
        spy=[[],[],[],[],[],[],[]]
        #print(response.text)
        for key, val in response.json()['Time Series (Daily)'].items():
            spy[0].append(float(val['1. open']))
            spy[1].append(float(val['2. high']))
            spy[2].append(float(val['3. low']))
            spy[3].append(float(val['4. close']))
            spy[4].append(float(val['5. adjusted close']))
            #spy[5].append(float(val['6. volume']))
            #spy[6].append(float(val['7. dividend amount']))
        t=list(response.json()['Time Series (Daily)'].keys())
        
        try:
            if (pd.to_datetime(t[0]).date()==pd.Timestamp.today().date())==False:
                x=get_intraday(ticker)
                spy[0].insert(0,float(x[0]))
                spy[1].insert(0,float(x[1]))
                spy[2].insert(0,float(x[2]))
                spy[3].insert(0,float(x[3]))
                spy[4].insert(0,float(x[4]))
                t.insert(0,str(pd.Timestamp.today().date()))

                #spy[0].append(float(t.insert(0,response2.json()[0]['date'])))
        except:
            print(ticker,'IEX')

        


        spy=pd.DataFrame({'Date':t,
                     'open':spy[0],
                      'high':spy[1],
                      'low':spy[2],
                      'close':spy[3],
                      'adj_close':spy[4]#,
                    #  'volume':spy[5],
                    #  'dividend':spy[6],
                     })
        spy=spy.astype({'Date': 'datetime64'})
        spy=spy.sort_values(by='Date')
        spy['highest_high']=spy.high.rolling(22).max()
        spy['lowest_low']=spy.low.rolling(22).min()
        spy['k']=(spy.close-spy.lowest_low)/(spy.highest_high-spy.lowest_low)
        spy['k']=spy.k.rolling(3).mean()
        spy['k_change'] = spy.k/spy.k.shift(3)-1
        res[0].append(ticker)
        res[1].append(spy.iloc[-1]['Date']) 
        res[2].append(spy.iloc[-1]['k']) 
        res[3].append(spy.iloc[-1]['k_change'])
        res[4].append(np.nanpercentile(spy.k_change,q=10))
        res[5].append(get_percentile(spy.k_change))
        temp_res=RSI(spy.adj_close)
        res[6].append(temp_res[0])
        res[7].append(temp_res[1])
        temp_res=macd_calcs(spy.adj_close)
        res[8].append(temp_res[0])
        res[9].append(temp_res[1])
        res[10].append(spy.adj_close.rolling(90).std().values[-1])
        res[11].append((spy.adj_close/spy.adj_close.shift(5)-1).rolling(4).mean().values[-1])
        res[12].append((spy.adj_close/spy.adj_close.shift(22)-1).values[-1])
        time.sleep(1)
        
    except Exception as e:
        exc_type, exc_obj, exc_tb = sys.exc_info()
        print(ticker,exc_type, exc_tb.tb_lineno)  


CRM


In [121]:
res=pd.DataFrame({'ticker':res[0], 
            'date':res[1],
             'k':res[2],
             'k_change':res[3],
             'k_10_percentile':res[4],
             'k_change_percentile':res[5],
             'RSI':res[6],
             'RSI_percentile':res[7],
             'hist_cons':res[8],    
             'hist_percentile':res[9],
             '90dayStd':res[10],
             'ret1wk_mean':res[11],
             '22dayret':res[12]    
             }
            )

In [122]:
res.sort_values(by='k_change',ascending=True)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret
0,CRM,2020-08-01,0.55864,0.122528,-0.445695,66.86,56.11,56.74,6,24.87,16.840932,0.019938,0.012299


In [34]:
res.sort_values(by='22dayret',ascending=False)

,ticker,date,k,k_change,k_10_percentile,k_change_percentile,RSI,RSI_percentile,hist_cons,hist_percentile,90dayStd,ret1wk_mean,22dayret
0,MELI,2020-08-01,0.868026,0.282924,-0.446197,77.24,69.1,87.34,6,99.05,194.037597,0.094502,0.147164


In [ ]:
res[(res.RSI_percentile>90) & (res.k_change>-100)].sort_values(by='ret1wk_mean',ascending=False)

In [ ]:
pk_444bcc6ef4a94b859405ef30fe66fa14 
https://cloud.iexapis.com/stable/stock/QQQ/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=1
    https://cloud.iexapis.com/stable/stock/CWH/intraday-prices?token=pk_9469e57bc642462d905e4dc0003523ab&chartIEXOnly=True&chartLast=30&chartIEXWhenNull=True

In [ ]:
if ~(pd.to_datetime(list(response.json()['Time Series (Daily)'].keys())[0])==pd.Timestamp.today()):
    d={'function': 'TIME_SERIES_INTRADAY','apikey':"YEUC2DL0F7KC43PB",'symbol':'MBUU','interval':'1min'}
    response = requests.get(
        'https://www.alphavantage.co/query',
        params=d,
        )

In [ ]:
d={'token':"pk_9469e57bc642462d905e4dc0003523ab",'chartIEXOnly':'True','chartLast':'1'}
response2 = requests.get(
        'https://cloud.iexapis.com/stable/stock/{}/intraday-prices?'.format('MBUU')#token={}&chartIEXOnly=True&chartLast=1'.format(holdings[0],'pk_9469e57bc642462d905e4dc0003523ab'),
        ,params=d,
        )

In [ ]:
t=pd.DataFrame(response.json()[0]['date'],response.json()[0]['open'],response.json()[0]['high'],response.json()[0]['low'],response.json()[0]['close'],response.json()[0]['close'])

In [ ]:
date=list(response.json()['Time Series (Daily)'].keys())

In [ ]:
date.insert(0,response2.json()[0]['date'])

In [ ]:
date

In [109]:
spy


,Date,open,high,low,close,adj_close,highest_high,lowest_low,k,k_change
1710,2013-10-16,38.00,39.6400,35.0600,37.16,37.16,NaN,NaN,NaN,NaN
1709,2013-10-17,37.13,42.0000,37.1000,41.60,41.60,NaN,NaN,NaN,NaN
1708,2013-10-18,43.70,44.9900,43.5900,44.39,44.39,NaN,NaN,NaN,NaN
1707,2013-10-21,49.00,49.0000,45.3900,46.24,46.24,NaN,NaN,NaN,NaN
1706,2013-10-22,47.96,48.0000,41.7680,42.95,42.95,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4,2020-07-28,255.39,256.0300,250.4401,251.01,251.01,263.63,228.81,0.670400,-0.163865
3,2020-07-29,253.56,261.8491,253.0300,259.53,259.53,263.63,228.81,0.742964,-0.007672
2,2020-07-30,258.96,265.3100,256.8600,261.94,261.94,265.31,229.80,0.808304,0.155701
1,2020-07-31,265.00,265.6800,259.8400,264.57,264.57,265.68,232.44,0.917985,0.369309


In [ ]:
np.nanpercentile(spy.k_change,q=10)

In [ ]:
response.json()[0]

In [ ]:
pd.to_datetime('today')

In [ ]:
pd.Timestamp.today().date()

In [ ]:
pd.to_datetime(t[0]).date()

In [ ]:
pd.to_datetime(t[0]).date()==pd.Timestamp.today().date()

In [ ]:
spy.adj_close.rolling(90).std().values[-1]

In [ ]:
(spy.adj_close/spy.adj_close.shift(22)-1).values[-1]

In [ ]:
(spy.adj_close/spy.adj_close.shift(5)-1).rolling(4).mean().values[-1]

In [130]:
spy=spy.set_index('Date')#['2020-07-01':]

In [131]:
spy['ret']=spy.adj_close/spy.adj_close.shift(5)-1

In [141]:
spy['ret22']=spy.adj_close/spy.adj_close.shift(22)-1

In [142]:
np.nanpercentile(spy.k_change,q=10),np.nanpercentile(spy.ret,q=10)

(-0.44569497601797003, -0.05896777368590391)

In [143]:
def each_percentile(x,val):
    return (x<val).sum()/len(x)

In [134]:
spy['ret_p']=spy.ret.apply(lambda x: each_percentile(spy.ret,x))

In [145]:
spy['ret22_p']=spy.ret.apply(lambda x: each_percentile(spy['ret22'],x))

In [146]:
spy['2020-07-01':]

,open,high,low,close,adj_close,highest_high,lowest_low,k,k_change,ret,ret_p,22dayret,ret22,ret22_p
Date,,,,,,,,,,,,,,
2020-07-01,188.10,192.820,187.4600,191.89,191.89,194.09,167.00,0.757352,0.097931,0.034169,0.744639,0.088059,0.088059,0.517624
2020-07-02,193.16,195.390,192.2900,192.53,192.53,195.39,167.00,0.856170,0.292366,0.022247,0.652699,0.105034,0.105034,0.473010
2020-07-06,194.44,199.450,193.8400,197.72,197.72,199.45,167.00,0.921579,0.417693,0.079434,0.912250,0.129893,0.129893,0.704954
2020-07-07,197.66,200.220,196.2100,196.38,196.38,200.22,167.00,0.910118,0.201711,0.071183,0.895982,0.145474,0.145474,0.668721
2020-07-08,198.00,201.200,197.3500,200.28,200.28,201.20,171.27,0.933452,0.090264,0.069129,0.889081,0.151829,0.151829,0.664530
2020-07-09,202.02,202.820,195.6400,200.74,200.74,202.82,171.27,0.929247,0.008321,0.046120,0.804535,0.137015,0.137015,0.567661
2020-07-10,200.06,200.730,197.0800,198.88,198.88,202.82,171.27,0.926151,0.017616,0.032982,0.735519,0.139322,0.139322,0.513187
2020-07-13,200.00,200.260,188.0300,188.34,188.34,202.82,171.27,0.783413,-0.160736,-0.047441,0.137787,0.034267,0.034267,0.231205
2020-07-14,185.99,189.960,183.3000,189.56,189.56,202.82,171.27,0.665293,-0.284051,-0.034729,0.191521,0.101773,0.101773,0.269657


In [148]:
spy['2020-02-20':'2020-3-31']


,open,high,low,close,adj_close,highest_high,lowest_low,k,k_change,ret,ret_p,22dayret,ret22,ret22_p
Date,,,,,,,,,,,,,,
2020-02-20,194.00,195.720,189.7700,193.36,193.36,195.720,177.30,0.903336,0.146975,0.020585,0.637663,0.061077,0.061077,0.465368
2020-02-21,191.84,192.040,186.7200,189.50,189.50,195.720,177.30,0.823675,0.000706,0.004559,0.487306,0.022832,0.022832,0.408676
2020-02-24,178.79,187.150,178.7900,185.94,185.94,195.720,177.30,0.667752,-0.248288,-0.021111,0.268425,0.017456,0.017456,0.310574
2020-02-25,188.47,189.980,179.4300,181.27,181.27,195.720,177.30,0.448969,-0.502988,-0.051389,0.123983,-0.014730,-0.014730,0.219374
2020-02-26,176.54,182.665,174.2500,178.87,178.87,195.720,174.25,0.299922,-0.635873,-0.072588,0.069509,-0.017791,-0.017791,0.165640
2020-02-27,172.21,178.330,170.0500,172.15,172.15,195.720,170.05,0.170839,-0.744158,-0.109692,0.025388,-0.047369,-0.047369,0.098102
2020-02-28,164.99,171.770,163.1200,170.40,170.40,195.720,163.12,0.173435,-0.613704,-0.100792,0.031797,-0.068089,-0.068089,0.112152
2020-03-02,172.20,176.790,168.5900,176.76,176.76,195.720,163.12,0.241175,-0.195874,-0.049371,0.131624,-0.027562,-0.027562,0.226276
2020-03-03,178.41,178.420,167.0900,169.37,169.37,195.720,163.12,0.277812,0.626158,-0.065648,0.082573,-0.087790,-0.087790,0.181908


So something can go down at random 
look to see how other stocks have done. if it is aleader like CWH or MELI NEED TO track and buy
that is part of the in-cycle market leader